In [ ]:
import pandas as pd

BASE_PATH = '/home/baldasseroniluisesteban/buckets/b1/datasets/'

df = pd.read_csv(BASE_PATH + 'competencia_02_fe.csv')
df.head()

In [ ]:
# Cargar el archivo con el diccionario de estadísticas (medias, cuartiles, deciles, etc.)
df_estadisticas = pd.read_csv(BASE_PATH + 'DiccionarioDatos_2024_nulos.csv')

# Cargar el archivo con las reglas de imputación
df_reglas_imputacion = pd.read_csv(BASE_PATH + 'rank.csv')

In [ ]:
def imputar_valores(df, df_estadisticas, df_reglas_imputacion):
    for _, row in df_reglas_imputacion.iterrows():
        columna = row['columna']
        regla = row['regla']
        
        # Obtener las estadísticas correspondientes de la columna
        stats = df_estadisticas[df_estadisticas['columna'] == columna].iloc[0]
        
        if regla == "media":
            valor_imputar = stats['media']
        elif regla == "mediana":
            valor_imputar = stats['mediana']
        elif regla == "cuartil_1":
            valor_imputar = stats['cuartil_1']
        elif regla == "cuartil_3":
            valor_imputar = stats['cuartil_3']
        elif regla == "deciles":
            valor_imputar = stats['deciles']  # Aquí tomarías un valor de deciles, ajusta según cómo se almacenen
        else:
            # Si la regla es algo no contemplado, por defecto usa la media
            valor_imputar = stats['media']
        
        # Imputar los valores faltantes
        df[columna].fillna(valor_imputar, inplace=True)
    
    return df


In [ ]:
# Aplicar la imputación
df = imputar_valores(df, df_estadisticas, df_reglas_imputacion)


In [ ]:
# Guardar el dataset imputado
df.to_csv('dataset_imputado.csv.gz', index=False, compression='gzip')
